<a href="https://colab.research.google.com/github/TasnimTamanna02/Detecting_Fake_News/blob/main/Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detecting Fake News with PassiveAggressiveClassifier


In [ ]:
import numpy as np
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projects/Detecting Fake News/news.csv')
df.sample(5)

In [ ]:
df.info()

In [ ]:
labels=df.label
labels.head()

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(df['text'], labels, test_size=0.2, random_state=5)

In [ ]:
x_train. sample(3)

In [ ]:
tf_idfvectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)
tf_train=tf_idfvectorizer.fit_transform(x_train)
tf_test=tf_idfvectorizer.transform(x_test)

In [ ]:
clf=PassiveAggressiveClassifier(max_iter=80)
clf.fit(tf_train,y_train)

In [ ]:
y_pred=clf.predict(tf_test)
print(classification_report(y_test,y_pred))

In [ ]:
confusion_matrix(y_test,y_pred, labels=['FAKE','REAL'])

In [46]:
!apt-get install git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
error: open("drive/.shortcut-targets-by-id/1MEH23sfuaCwruyPYSvAcaoXXFsrahO7X/Study materials /(Re.Pa) Affix Array.gdoc"): Operation not supported
error: unable to index file 'drive/.shortcut-targets-by-id/1MEH23sfuaCwruyPYSvAcaoXXFsrahO7X/Study materials /(Re.Pa) Affix Array.gdoc'
fatal: adding files failed
Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Nam